In [1]:
!pip install avalanche-lib==0.6

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.2/134.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.4/993.4 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.2/585.2 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.4/926.4 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.6/558.6 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 979.8/979.8 kB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.9/505.9 kB 23.9 MB/s eta 0:00:00
  

In [2]:
import torch
import torch.nn as nn
from torchvision.transforms import Compose, ToTensor, Normalize
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss
from torch.optim import SGD
# from avalanche.benchmarks.generators import nc_benchmark
from avalanche.benchmarks import nc_benchmark
from avalanche.training.supervised import GDumb
from avalanche.logging import InteractiveLogger
from avalanche.training.plugins import EvaluationPlugin
from avalanche.evaluation.metrics import accuracy_metrics


In [3]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=False),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=False),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.fc1 = nn.Linear(32 * 16 * 16, 2048)
        self.fc2 = nn.Linear(2048, 512)
        self.fc3 = nn.Linear(512, 128)
        self.fc4 = nn.Linear(128, 32)
        self.fc5 = nn.Linear(32, 10)

    def forward(self, x):
        x = self.conv(x)
        x = x.view(-1, 32 * 16 * 16)
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.fc2(x)
        x = torch.relu(x)
        x = self.fc3(x)
        x = torch.relu(x)
        x = self.fc4(x)
        x = torch.relu(x)
        x = self.fc5(x)
        x = torch.log_softmax(x, dim=1)
        return x


In [4]:
def benchmark_creation():
    # Transformations
    transform = Compose([
        ToTensor(),
        Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    # Load CIFAR-10 dataset
    train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)
    test_dataset = CIFAR10(root='./data', train=False, download=True, transform=transform)

    # Generate the benchmark
    generic_scenario = nc_benchmark(
        train_dataset,
        test_dataset,
        n_experiences=5,
        task_labels=True,
        shuffle=True,
        seed=42
    )
    return generic_scenario


In [5]:
def main():
    # Config
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # Benchmark Creation
    benchmark = benchmark_creation()

    # Prepare for Training & Testing
    train_stream = benchmark.train_stream
    test_stream = benchmark.test_stream

    # Model Creation
    model = CNN()

    # Optimizer
    optimizer = SGD(model.parameters(), lr=0.01)

    # CL strategy
    criterion = CrossEntropyLoss()

    # Logger
    interactive_logger = InteractiveLogger()

    # Evaluation Plugin
    eval_plugin = EvaluationPlugin(
        accuracy_metrics(
            minibatch=True,
            epoch=True,
            epoch_running=True,
            experience=True,
            stream=True,
        ),
        loggers=[interactive_logger],
        collect_all=True
    )

    # Strategy Instance
    strategy = GDumb(
        model=model,
        optimizer=optimizer,
        criterion=criterion,
        mem_size=16384,
        train_mb_size=32,
        train_epochs=100,
        eval_mb_size=32,
        device=device,
        evaluator=eval_plugin,
    )

    # Training Loop
    print("Starting experiment...\n")
    results = []
    for i, experience in enumerate(benchmark.train_stream):
        print("Start of experience #:", experience.current_experience)
        print("Current Classes: ", experience.classes_in_this_experience, "\n")

        # Train
        strategy.train(experience, eval_streams=[benchmark.test_stream])
        print("Training completed\n")

        # Test
        results.append(strategy.eval(benchmark.test_stream))
        print("Evaluation completed\n\n\n\n\n")

    print(f"Test metrics:\n{results}")

    all_metrics = strategy.evaluator.get_all_metrics()
    print(f"Stored metrics: {list(all_metrics.keys())}")


In [ ]:
if __name__ == "__main__":
    main()


100%|██████████| 170M/170M [00:05<00:00, 33.4MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/avalanche/training/plugins/gdumb.py:47: DeprecationWarning: Call to deprecated function update (removal in version 0.7: switch to pre_adapt and post_adapt)
  self.storage_policy.update(strategy, **kwargs)


Starting experiment...

Start of experience #: 0
Current Classes:  [2, 6] 

-- >> Start of training phase << --
100%|██████████| 313/313 [00:07<00:00, 40.61it/s]
Epoch 0 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.6543
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.9375
	Top1_RunningAcc_Epoch/train_phase/train_stream/Task000 = 0.6543
100%|██████████| 313/313 [00:06<00:00, 45.90it/s]
Epoch 1 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.8103
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.8125
	Top1_RunningAcc_Epoch/train_phase/train_stream/Task000 = 0.8103
100%|██████████| 313/313 [00:06<00:00, 49.44it/s]
Epoch 2 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.8415
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.8125
	Top1_RunningAcc_Epoch/train_phase/train_stream/Task000 = 0.8415
100%|██████████| 313/313 [00:07<00:00, 42.84it/s]
Epoch 3 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.8663
	Top1_Acc_MB/train_phase/train_strea